### Import the required packages into python

In [ ]:
import pulp
import itertools

### Set up the Deterministic minimisation problem

In [1]:
def initialise_deterministic_minimisation_problem(
    specialties, hospitals, bands
):
    """
    Initialise the mininmisation problem. 
    Set decision variables for the models.
    """
    sh = [(s,h) for s in specialties for h in hospitals]
    shb = [(s,h,b) for s in specialties for h in hospitals for b in bands]

    prob = pulp.LpProblem("Deterministic", pulp.LpMinimize)
    
    xbed = pulp.LpVariable.dicts(
        "Xbed", (specialties, hospitals), lowBound=0, cat='Integer'
    )
    xstaff = pulp.LpVariable.dicts(
        "Xstaff", (specialties, hospitals, bands), lowBound=0,cat='Integer'
    )
    return prob, sh, shb, xbed, xstaff

def add_deterministic_constraints(xbed, xstaff, UBbed, UBstaff, D, K, R, sh, shb, prob):
    """
    Add the constraints that are required for the deterministic model
        
    - Constraints 1-6: Ensures demand is met across all specialties and all regions
    - Constraint 7: Ensures beds are only able to open in a ward if the facilities are able to be opened 
    - Constraint 8: Ensures staffing ratios are met
    - Constraint 9: Ensures beds deployed does not exceed maximum capacity of hospital
    - Constraint 10: Ensures staff deployed does not exceed maximum staffing resources
    """
    
    for s in specialties:
        prob += pulp.lpSum(xbed[s][h] for h in region1) >= pulp.lpSum(D[s][0]) #Constraint 1
        prob += pulp.lpSum(xbed[s][h] for h in region2) >= pulp.lpSum(D[s][1]) #Constraint 2
        prob += pulp.lpSum(xbed[s][h] for h in region3) >= pulp.lpSum(D[s][2]) #Constraint 3
        prob += pulp.lpSum(xbed[s][h] for h in region4) >= pulp.lpSum(D[s][3]) #Constraint 4
        prob += pulp.lpSum(xbed[s][h] for h in region5) >= pulp.lpSum(D[s][4]) #Constraint 5
        prob += pulp.lpSum(xbed[s][h] for h in region6) >= pulp.lpSum(D[s][5]) #Constraint 6
        
        for h in hospitals:
            prob += pulp.lpSum(xbed[s][h])<= pulp.lpSum(K[s][h]) # Constraint 7
                
            for b in bands:
                prob += pulp.lpSum(xstaff[s][h][b]) >= pulp.lpSum(R[s][b]*(xbed[s][h])) #Constraint 8
            
    for h in hospitals:
        prob += pulp.lpSum(xbed[s][h] for s in specialties) <= UBbed[h]  #Constraint 9
        
    for b in bands:
        prob += pulp.lpSum(xstaff[s][h][b] for (s,h) in sh) <= UBstaff[b] # #Constraint 10
        
    return prob

def solve_deterministic_minimisation_problem(
    specialties,
    bands,
    region1,
    region2,
    region3,
    region4,
    region5,
    region6,
    hospitals,
    regions,
    D,
    K,
    R,
    cbed,
    cstaff,
    UBbed,
    UBstaff,
):
    """
    Solves the deterministic problem, with the objective function being minimised
    Uses the CBC_CMD Solver, with maximum run time of 60 seconds and precision of 0.01
    """
    prob, sh, shb, xbed, xstaff = initialise_deterministic_minimisation_problem(
        specialties=specialties, 
        hospitals=hospitals, 
        bands=bands
    )
    
    prob += (
        pulp.lpSum((xbed[s][h] * cbed[s][h]) for (s,h) in sh) +
        pulp.lpSum((xstaff[s][h][b]*cstaff[b]) for (s,h,b) in shb)
    )
    
    prob = add_deterministic_constraints(
        xbed=xbed, 
        xstaff=xstaff, 
        UBbed=UBbed, 
        UBstaff=UBstaff,
        D=D, 
        K=K,
        R=R,
        sh=sh,
        shb=shb,
        prob=prob,
    )
    prob.solve(pulp.PULP_CBC_CMD(maxSeconds=60,fracGap=0.01))
    
    return prob

### The following cell can be edited to allow the user to enter their own parameters and values for the model

In [ ]:
specialties = list(itertools.chain(range(0, ))) #Creates list of specialties
bands = list(itertools.chain(range(0, ))) #Creates list of nursing bands
regions = list(itertools.chain(range(0, ))) #Creates List of regions

region1 = []
region2 = []
region3 = []
region4 = []
region5 = []
region6 = []
hospitals = region1 + region2 +region3 + region4 +region5 +region6
D = [ ,
[],
]
K = [
[],
]
R = [
[],
] 
cbed = [
[],
]
cstaff = []
UBstaff = []
UBbed =[]

### The following runs the deterministic model

In [ ]:
prob = solve_deterministic_minimisation_problem(
    specialties,
    bands,
    region1,
    region2,
    region3,
    region4,
    region5,
    region6,
    hospitals,
    regions,
    D,
    K,
    R,
    cbed,
    cstaff,
    UBbed,
    UBstaff,
)

### The following outputs the results of the deterministic model

In [ ]:
print("Solution Status = ", pulp.LpStatus[prob.status])
print("Total price = ", pulp.value(prob.objective))  
for v in prob.variables():
    if v.varValue >= 0:
        #print(v.varValue)
        print(v.name, "=", v.varValue)
production = [v.varValue for v in prob.variables()]